In [303]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [285]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from yahoo_fin import options
import numpy as np
import math
from scipy.stats import norm
import datetime
from sklearn.model_selection import train_test_split
import yahoo_fin.stock_info as si

In [242]:
def blackScholes(spot_price,strike_price,risk_free_rate,time_to_maturity,volatility,option_type):
    time_to_maturity = time_to_maturity/365
    d1 = ((spot_price / strike_price).apply(math.log)+ (risk_free_rate + 0.5 * volatility**2) * time_to_maturity) / (volatility * (time_to_maturity.apply(math.sqrt)))
    d2 = d1 - volatility * time_to_maturity.apply(math.sqrt)    
    # Calculate option price
    if option_type == 'call':
        option_price = spot_price * d1.apply(norm.cdf) - strike_price * (-risk_free_rate * time_to_maturity).apply(math.exp) * d2.apply(norm.cdf)
    else:
        option_price = strike_price * (-risk_free_rate * time_to_maturity).apply(math.exp) * -d2.apply(norm.cdf) - spot_price * -d1.apply(norm.cdf)
    return option_price

def BlackScholesClassifier(bs_price,market_data_price):
    if(abs(bs_price - market_data_price)/bs_price <=0.05): # Accurate
        return 0
    elif bs_price > market_data_price: # Overpriced
        return 1
    else: 
        return -1
    
def getAnnualVolatility(ticker):
    yearlyPrices = si.get_data(ticker,start_date = pd.to_datetime("today") - pd.offsets.DateOffset(years=1) ,end_date =pd.to_datetime("today") )
    yearReturns = yearlyPrices["close"].pct_change()
    dailyVolatility= yearReturns.std()
    return math.sqrt(len(yearlyPrices))*dailyVolatility


In [213]:
ticker = 'AAPL'
interestRateOneYearUSTreasury = 0.05
price = si.get_live_price(ticker)
expirationDates = options.get_expiration_dates(ticker)


In [222]:
liquidOptions = pd.DataFrame(columns=["Contract Name","Last Trade Date","Strike","Last Price","Bid","Ask","Change","% Change","Volume","Open Interest","Implied Volatility","MaturityDate","Today","StockPrice","Underlying","AnnualVol"])
for optionExpiryDate in expirationDates:
    callData = options.get_calls(ticker,date = optionExpiryDate)
    callData = callData[ callData["Volume"].apply(str).str.isnumeric() ]
    callData["Volume"] = callData["Volume"].apply(int)
    callData["Today"] = pd.to_datetime("today")
    callData["StockPrice"] = price
    callData["AnnualVol"] = getAnnualVolatility(ticker)
    callData["MaturityDate"] = pd.to_datetime(optionExpiryDate)
    callData["Underlying"] = ticker
    totalVolume = sum(callData["Volume"])
    liquidOptions = pd.concat([liquidOptions,pd.DataFrame(callData[callData["Volume"]/totalVolume >= 0.1],columns=liquidOptions.columns)])

liquidOptions["TreasuryRate"] = interestRateOneYearUSTreasury   
liquidOptions["DaysToMaturity"] = (liquidOptions["MaturityDate"] - liquidOptions["Today"])/np.timedelta64(1, 'D')
liquidOptions["BSPrice"]= blackScholes(liquidOptions["StockPrice"],liquidOptions["Strike"],liquidOptions["TreasuryRate"],liquidOptions["DaysToMaturity"],liquidOptions["AnnualVol"],"call")
#print(liquidOptions[["Underlying","StockPrice","Strike","Volume","DaysToMaturity","AnnualVol","BSPrice","Last Price"]])


In [243]:
liquidOptions["RelativePriceDiff"] = (liquidOptions["Last Price"] - liquidOptions["BSPrice"]).apply(abs) / liquidOptions["BSPrice"]
liquidOptions["BSClassification"] = liquidOptions.apply(lambda x : BlackScholesClassifier(x["BSPrice"],x["Last Price"]),axis=1)
print(liquidOptions[["Underlying","StockPrice","Strike","DaysToMaturity","AnnualVol","BSPrice","Last Price","RelativePriceDiff","isAccurate"]].head())

   Underlying  StockPrice Strike  DaysToMaturity AnnualVol   BSPrice  \
14       AAPL  191.240005  190.0        4.213827  0.217207  2.530126   
15       AAPL  191.240005  195.0        4.213827  0.217207  0.532533   
31       AAPL  191.240005  190.0       11.213807  0.217207  3.722943   
32       AAPL  191.240005  192.5       11.213807  0.217207  2.454919   
33       AAPL  191.240005  195.0       11.213807  0.217207  1.519646   

   Last Price RelativePriceDiff  isAccurate  
14        2.4          0.051431           0  
15       0.33           0.38032           0  
31        3.3          0.113604           0  
32       1.89          0.230117           0  
33       0.96          0.368274           0  


In [320]:
model = Sequential()

model.add(Dense(128,input_shape=(5,)))

model.add(Dense(64,activation="relu"))

model.add(Dense(32,activation="relu"))

model.add(Dense(3,activation="softmax"))

In [329]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])
model_input = liquidOptions[["StockPrice","Strike","DaysToMaturity","TreasuryRate","AnnualVol"]]
model_input = np.asarray(model_input_train).astype(np.float64)
model_output = liquidOptions["BSClassification"]
model_output = pd.get_dummies(pd.Categorical(model_output))

earlyStopping = EarlyStopping(monitor='val_loss',patience=20)
modelToSave = ModelCheckpoint('best_model.keras',save_best_only=True) 

X_train, X_test, y_train, y_test = train_test_split(model_input, model_output, test_size=0.2)

fittedModel = model.fit(X_train,y_train,epochs=200,validation_data=(X_test,y_test),callbacks=[earlyStopping,modelToSave])


Epoch 1/200
2/2 [==============================] - 0s 65ms/step - loss: 1.7580 - accuracy: 0.8684 - val_loss: 1.0061 - val_accuracy: 0.9000
Epoch 2/200
2/2 [==============================] - 0s 10ms/step - loss: 1.0631 - accuracy: 0.8158 - val_loss: 2.0444 - val_accuracy: 0.8000
Epoch 3/200
2/2 [==============================] - 0s 17ms/step - loss: 2.2004 - accuracy: 0.7632 - val_loss: 0.5280 - val_accuracy: 0.9000
Epoch 4/200
2/2 [==============================] - 0s 17ms/step - loss: 0.2835 - accuracy: 0.9474 - val_loss: 0.1607 - val_accuracy: 0.9000
Epoch 5/200
2/2 [==============================] - 0s 10ms/step - loss: 0.9610 - accuracy: 0.8421 - val_loss: 0.5654 - val_accuracy: 0.9000
Epoch 6/200
2/2 [==============================] - 0s 11ms/step - loss: 0.3444 - accuracy: 0.9211 - val_loss: 1.5482 - val_accuracy: 0.9000
Epoch 7/200
2/2 [==============================] - 0s 11ms/step - loss: 0.5799 - accuracy: 0.8947 - val_loss: 2.0573 - val_accuracy: 0.8000
Epoch 8/200
2/2 [===

In [335]:
accuracy = model.evaluate(X_test,y_test)[1]
print(accuracy)

1/1 [==============================] - 0s 16ms/step - loss: 0.9647 - accuracy: 0.9000
0.8999999761581421
